In [1]:
import pandas as pd
import numpy as np


# download config_bdd30-100.pkl here 
# https://filesender.renater.fr/?s=download&token=556ae671-4546-493e-f96f-3cd3c1919724#

# TODO: excellent "test" set since 15K and 30K have not been subject to specialization
# pd.read_csv('config_bdd15000-20000.csv') 
# pd.read_csv('config_bdd20000-30000.csv') 
# 
rawtuxdata = pd.read_csv('../tuxml-datasets/config_bdd100000-1265000.csv') # pd.read_pickle('config_bdd30-100.pkl')

basic_head = ["cid", "time", "date"] # "compile"
size_methods = ["vmlinux", "GZIP-bzImage", "GZIP-vmlinux", "GZIP", "BZIP2-bzImage", 
              "BZIP2-vmlinux", "BZIP2", "LZMA-bzImage", "LZMA-vmlinux", "LZMA", "XZ-bzImage", "XZ-vmlinux", "XZ", 
              "LZO-bzImage", "LZO-vmlinux", "LZO", "LZ4-bzImage", "LZ4-vmlinux", "LZ4"]

print("some configurations may have X86_32 (coz we have tested/tried some options and there are in the database) -- we removed them of course")
# we only keep X86_64 configurations
#rawtuxdata = rawtuxdata[rawtuxdata['X86_64'] == 'y'] ### TODO: I've impression it's not the most effective way (wrt memory) to filter 
if 'X86_64' in rawtuxdata.columns:
    print(rawtuxdata['X86_64'].describe())
    rawtuxdata.query("X86_64 == 'y'", inplace=True) # only keep them


/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (1150,6015,6026,6717,7350,7676,7726) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


some configurations may have X86_32 (coz we have tested/tried some options and there are in the database) -- we removed them of course
count     26751
unique        1
top           y
freq      26751
Name: X86_64, dtype: object


In [2]:
### TODO: ensure that the same configuration leads to the same compilation status (normally it's the case)
# remove entries with same configurations
print(str(len(rawtuxdata)) + " before the removal of some entries (those with same configurations)")
# tuxdata.drop_duplicates(subset=tuxdata.columns.difference(['vmlinux']), inplace=True)
rawtuxdata.drop_duplicates(subset=rawtuxdata.columns.difference(size_methods).difference(basic_head), inplace=True)
print(str(len(rawtuxdata)) + " after the removal of some entries (those with same configurations)")

26751 before the removal of some entries (those with same configurations)
26730 after the removal of some entries (those with same configurations)


In [3]:
cid_to_remove=74459 # compilation time is very low (2 sec)
cid_to_remove2=30698 # size is 0! 

rawtuxdata = rawtuxdata[rawtuxdata.cid != cid_to_remove]
rawtuxdata = rawtuxdata[rawtuxdata.cid != cid_to_remove2]

In [4]:
# assert (len(rawtuxdata) == 20001)
len(rawtuxdata)

26730

In [5]:
len(rawtuxdata.query("vmlinux == -1").index)

113

In [6]:
def nbyes(row):
    return sum(row == "y")                    
rawtuxdata['nbyes'] = rawtuxdata.apply(nbyes, axis=1)

In [7]:
rawtuxdata.sort_values(by="nbyes")[:10]

,cid,date,time,vmlinux,GZIP-bzImage,GZIP-vmlinux,GZIP,BZIP2-bzImage,BZIP2-vmlinux,BZIP2,...,APDS9960,ARCH_SUPPORTS_INT128,SLABINFO,MICROCODE_AMD,ISDN_DRV_HISAX,CHARGER_BQ24190,SND_SOC_NAU8825,BH1750,NETWORK_FILESYSTEMS,nbyes
24973,124973,2018-10-03 17:08:04,27.3111,7317008,646608,2733176,501221,4722128,6808144,458492,...,n,y,n,n,n,n,n,n,n,240
24977,124978,2018-10-03 17:58:42,26.4772,7304656,728528,2815096,584622,4791760,6877776,528161,...,n,y,n,n,n,n,n,n,n,241
15110,115110,2018-08-14 17:46:43,243.4590,10486184,1991632,4077008,1844458,6054864,8139688,1788609,...,n,y,n,n,n,n,n,m,n,636
19649,119649,2018-09-26 00:04:28,162.7380,10707816,1978320,4064376,1835908,6082512,8168016,1819382,...,n,y,n,n,n,n,n,m,n,712
15655,115655,2018-08-15 03:27:22,722.3290,10504040,1835984,3921016,1690306,5907408,7991888,1644649,...,n,y,n,n,n,n,n,n,n,721
25417,125418,2018-10-05 03:38:25,221.2110,10511608,1983440,4068816,1836771,6091728,8176552,1826593,...,n,y,n,n,n,n,n,n,n,724
14626,114626,2018-08-14 15:07:08,187.8910,13181216,2514896,4601296,2366604,6569936,8655784,2306560,...,n,y,n,n,n,n,n,m,n,725
21374,121374,2018-09-27 14:00:36,74.0362,11373168,2225104,4310480,2075211,6321104,8405928,2054958,...,n,y,n,n,n,n,n,m,n,727
4628,104628,2018-08-06 12:46:04,192.1140,10916160,2351056,4437456,2205581,6442960,8528808,2178659,...,n,y,n,y,n,n,n,n,n,728
19913,119913,2018-09-26 02:19:24,91.8989,10767440,1946576,4031952,1796621,6042576,8127400,1775203,...,n,y,n,y,n,m,n,n,n,736


In [8]:
# tinykernel
rawtuxdata.query("vmlinux == 7317008") #.iloc[0]

,cid,date,time,vmlinux,GZIP-bzImage,GZIP-vmlinux,GZIP,BZIP2-bzImage,BZIP2-vmlinux,BZIP2,...,APDS9960,ARCH_SUPPORTS_INT128,SLABINFO,MICROCODE_AMD,ISDN_DRV_HISAX,CHARGER_BQ24190,SND_SOC_NAU8825,BH1750,NETWORK_FILESYSTEMS,nbyes
24973,124973,2018-10-03 17:08:04,27.3111,7317008,646608,2733176,501221,4722128,6808144,458492,...,n,y,n,n,n,n,n,n,n,240


In [9]:
rawtuxdata.query("GENERIC_ALLOCATOR == 'y'") # & DRM_VBOXVIDEO == 'y'")
#rawtuxdata.query("AIC7XXX_BUILD_FIRMWARE == 'y'")
#rawtuxdata.query("X86_64 == 'y'")

,cid,date,time,vmlinux,GZIP-bzImage,GZIP-vmlinux,GZIP,BZIP2-bzImage,BZIP2-vmlinux,BZIP2,...,APDS9960,ARCH_SUPPORTS_INT128,SLABINFO,MICROCODE_AMD,ISDN_DRV_HISAX,CHARGER_BQ24190,SND_SOC_NAU8825,BH1750,NETWORK_FILESYSTEMS,nbyes
0,100000,2018-08-03 20:33:51,681.5390,42045136,13280208,15365592,13132675,16851920,18936744,12584626,...,n,y,n,n,n,y,n,n,n,3181
1,100001,2018-08-03 20:34:03,678.4830,24805528,6431696,8516728,6284566,10322896,12407376,6059997,...,m,y,n,n,n,n,m,n,y,1575
2,100002,2018-08-03 20:34:14,747.4100,21632608,4812752,6899152,4665550,8822736,10908584,4558223,...,n,y,n,n,n,m,n,n,n,1519
3,100003,2018-08-03 20:34:38,381.0910,20133672,3556816,5641680,3409988,7558608,9642920,3292132,...,n,y,y,y,n,n,m,n,y,1141
4,100004,2018-08-03 20:35:05,272.8070,19522472,5289424,7374288,5139561,9295312,11379624,5027510,...,y,y,n,n,n,n,n,y,n,1920
6,100006,2018-08-03 20:36:10,489.5340,19217936,4305872,6390904,4162356,8324048,10408528,4062711,...,m,y,y,n,n,m,n,n,n,1500
7,100007,2018-08-03 20:37:22,552.0210,19295120,5895632,7980152,5748871,9754064,11838032,5488559,...,n,y,n,n,n,n,n,n,n,1305
8,100008,2018-08-03 20:37:25,1744.8700,22402592,7066064,9151952,6920219,10998224,13083560,6732436,...,n,y,n,y,n,y,n,n,n,1937
9,100009,2018-08-03 20:37:44,597.4580,33691800,10024720,12111408,9856772,13874960,15961096,9591811,...,n,y,n,n,n,y,n,n,n,2344
10,100010,2018-08-03 20:37:57,468.9510,19557872,4417296,6503984,4248515,8427280,10513416,4141049,...,m,y,y,y,n,n,n,m,n,1421


In [10]:
rawtuxdata.query("vmlinux != -1").sort_values(by="vmlinux")[:20]

,cid,date,time,vmlinux,GZIP-bzImage,GZIP-vmlinux,GZIP,BZIP2-bzImage,BZIP2-vmlinux,BZIP2,...,APDS9960,ARCH_SUPPORTS_INT128,SLABINFO,MICROCODE_AMD,ISDN_DRV_HISAX,CHARGER_BQ24190,SND_SOC_NAU8825,BH1750,NETWORK_FILESYSTEMS,nbyes
24977,124978,2018-10-03 17:58:42,26.4772,7304656,728528,2815096,584622,4791760,6877776,528161,...,n,y,n,n,n,n,n,n,n,241
24973,124973,2018-10-03 17:08:04,27.3111,7317008,646608,2733176,501221,4722128,6808144,458492,...,n,y,n,n,n,n,n,n,n,240
23091,123091,2018-09-29 15:53:30,95.9230,8716424,1712080,3798136,1568601,5779408,7864912,1517048,...,n,y,n,n,n,n,n,m,n,741
15110,115110,2018-08-14 17:46:43,243.4590,10486184,1991632,4077008,1844458,6054864,8139688,1788609,...,n,y,n,n,n,n,n,m,n,636
15655,115655,2018-08-15 03:27:22,722.3290,10504040,1835984,3921016,1690306,5907408,7991888,1644649,...,n,y,n,n,n,n,n,n,n,721
7038,107038,2018-08-07 12:43:05,446.9600,10504144,2044368,4129912,1897781,6128080,8213072,1864908,...,m,y,n,n,n,n,n,m,n,847
25417,125418,2018-10-05 03:38:25,221.2110,10511608,1983440,4068816,1836771,6091728,8176552,1826593,...,n,y,n,n,n,n,n,n,n,724
12521,112521,2018-08-11 12:28:09,230.6000,10648928,2634704,4720080,2484960,6689744,8774568,2424357,...,m,y,n,n,n,n,n,n,n,835
19367,119367,2018-09-25 21:43:18,220.9510,10681376,2183120,4269520,2035262,6279120,8364968,2016228,...,m,y,n,n,n,n,n,m,n,841
19649,119649,2018-09-26 00:04:28,162.7380,10707816,1978320,4064376,1835908,6082512,8168016,1819382,...,n,y,n,n,n,n,n,m,n,712


In [11]:
compilation_failures = len(rawtuxdata.query("vmlinux == -1").index)
compilation_successes = len(rawtuxdata.query("vmlinux != -1").index)
n_compilations = len(rawtuxdata.index)
assert(compilation_successes + compilation_failures == n_compilations)
compilation_failure_percentage = (compilation_failures / n_compilations) * 100
print("compilation failures:", compilation_failures, "out of", n_compilations, "(", compilation_failure_percentage, "%)")

compilation failures: 113 out of 26730 ( 0.4227459783015339 %)


In [12]:
# BUGS EXPLORATION
def bug_exploration():
    #rawtuxdata.query("AIC7XXX_BUILD_FIRMWARE == 'y'")[['cid', 'vmlinux']]
    #rawtuxdata.query("AIC79XX_BUILD_FIRMWARE == 'y'")[['cid', 'vmlinux']]
    #rawtuxdata.query("WANXL_BUILD_FIRMWARE == 'y'")[['cid', 'vmlinux']]
    #rawtuxdata.query("GENERIC_ALLOCATOR == 'n' & DRM_VBOXVIDEO == 'y'")[['cid', 'vmlinux']]
    #rawtuxdata.query("GENERIC_ALLOCATOR == 'y' & DRM_VBOXVIDEO == 'y'")[['cid', 'vmlinux']]
    #rawtuxdata.query("GENERIC_ALLOCATOR == 'n' & DRM_VBOXVIDEO == 'm'")[['cid', 'vmlinux']]
    return rawtuxdata.query("DRM_VBOXVIDEO == 'y'")[['cid', 'vmlinux']]

#bug_exploration()


# y or n values
n_AIC7XXX_alone = len(rawtuxdata.query("AIC7XXX_BUILD_FIRMWARE == 'y'").index)
n_AIC79XX_alone = len(rawtuxdata.query("AIC79XX_BUILD_FIRMWARE == 'y'").index)
# masking effects!
n_AIC_both = len(rawtuxdata.query("AIC79XX_BUILD_FIRMWARE == 'y' & AIC7XXX_BUILD_FIRMWARE == 'y'").index)
n_AIC_or = len(rawtuxdata.query("AIC7XXX_BUILD_FIRMWARE == 'y' | AIC79XX_BUILD_FIRMWARE == 'y'").index)
print("AIC7XXX_BUILD_FIRMWARE (without 79XX)", n_AIC7XXX_alone - n_AIC_both, "AIC79XX_BUILD_FIRMWARE (without X7XX)", n_AIC79XX_alone - n_AIC_both)
print("AIC7XXX_BUILD_FIRMWARE or AIC79XX_BUILD_FIRMWARE",  n_AIC_or)
print("Without the 2 major configuration bugs, caused by 2 individual options, it remains", len(rawtuxdata.query("AIC7XXX_BUILD_FIRMWARE != 'y' & AIC79XX_BUILD_FIRMWARE != 'y' & vmlinux == -1").index))
# masking effects!
# option WANXL_BUILD_FIRMWARE responsible of many bugs, except... 1
n_wanxl = len(rawtuxdata.query("WANXL_BUILD_FIRMWARE == 'y' & (AIC7XXX_BUILD_FIRMWARE != 'y' & AIC79XX_BUILD_FIRMWARE != 'y')").index)
print("WANXL_BUILD_FIRMWARE", n_wanxl)
# same as previously, but we only keep compilation failures... and surprise:
n_wanxl_failure = len(rawtuxdata.query("WANXL_BUILD_FIRMWARE == 'y' & (AIC7XXX_BUILD_FIRMWARE != 'y' & AIC79XX_BUILD_FIRMWARE != 'y' & vmlinux == -1)").index)
print("There is apparently one subtle case with WANXL_BUILD_FIRMWARE=y causing **NO** configuration bug (with VIDEO_RCAR_VIN)", n_wanxl_failure) # len(rawtuxdata.query("GENERIC_ALLOCATOR == 'y' & vmlinux == -1").index)

assert ((compilation_failures - (n_AIC_or + n_wanxl_failure)) == len(rawtuxdata.query("WANXL_BUILD_FIRMWARE != 'y' & AIC7XXX_BUILD_FIRMWARE != 'y' & AIC79XX_BUILD_FIRMWARE != 'y' & vmlinux == -1").index))
print("still tracking", (compilation_failures - (n_AIC_or + n_wanxl_failure)), "configuration bugs")
print("let's now investigate GENERIC_ALLOCATOR")
len(rawtuxdata.query("GENERIC_ALLOCATOR == 'y' & WANXL_BUILD_FIRMWARE != 'y' & AIC7XXX_BUILD_FIRMWARE != 'y' & AIC79XX_BUILD_FIRMWARE != 'y' & vmlinux == -1").index), len(rawtuxdata.query("GENERIC_ALLOCATOR == 'n' & WANXL_BUILD_FIRMWARE != 'y' & AIC7XXX_BUILD_FIRMWARE != 'y' & AIC79XX_BUILD_FIRMWARE != 'y' & vmlinux == -1").index)
#len(rawtuxdata.query("WANXL_BUILD_FIRMWARE != 'y' & AIC7XXX_BUILD_FIRMWARE != 'y' & AIC79XX_BUILD_FIRMWARE != 'y' & vmlinux == -1").index)
#len(rawtuxdata.query("vmlinux == -1").index)

#n_AIC_or + n_wanxl

AIC7XXX_BUILD_FIRMWARE (without 79XX) 0 AIC79XX_BUILD_FIRMWARE (without X7XX) 0
AIC7XXX_BUILD_FIRMWARE or AIC79XX_BUILD_FIRMWARE 0
Without the 2 major configuration bugs, caused by 2 individual options, it remains 113
WANXL_BUILD_FIRMWARE 1
There is apparently one subtle case with WANXL_BUILD_FIRMWARE=y causing **NO** configuration bug (with VIDEO_RCAR_VIN) 1
still tracking 112 configuration bugs
let's now investigate GENERIC_ALLOCATOR


(19, 93)

In [13]:
rawtuxdata.sort_values(by='cid')

,cid,date,time,vmlinux,GZIP-bzImage,GZIP-vmlinux,GZIP,BZIP2-bzImage,BZIP2-vmlinux,BZIP2,...,APDS9960,ARCH_SUPPORTS_INT128,SLABINFO,MICROCODE_AMD,ISDN_DRV_HISAX,CHARGER_BQ24190,SND_SOC_NAU8825,BH1750,NETWORK_FILESYSTEMS,nbyes
0,100000,2018-08-03 20:33:51,681.5390,42045136,13280208,15365592,13132675,16851920,18936744,12584626,...,n,y,n,n,n,y,n,n,n,3181
1,100001,2018-08-03 20:34:03,678.4830,24805528,6431696,8516728,6284566,10322896,12407376,6059997,...,m,y,n,n,n,n,m,n,y,1575
2,100002,2018-08-03 20:34:14,747.4100,21632608,4812752,6899152,4665550,8822736,10908584,4558223,...,n,y,n,n,n,m,n,n,n,1519
3,100003,2018-08-03 20:34:38,381.0910,20133672,3556816,5641680,3409988,7558608,9642920,3292132,...,n,y,y,y,n,n,m,n,y,1141
4,100004,2018-08-03 20:35:05,272.8070,19522472,5289424,7374288,5139561,9295312,11379624,5027510,...,y,y,n,n,n,n,n,y,n,1920
5,100005,2018-08-03 20:35:45,352.1530,14360096,3450640,5537328,3282754,7530256,9616392,3245966,...,n,y,n,n,n,n,n,m,n,1152
6,100006,2018-08-03 20:36:10,489.5340,19217936,4305872,6390904,4162356,8324048,10408528,4062711,...,m,y,y,n,n,m,n,n,n,1500
7,100007,2018-08-03 20:37:22,552.0210,19295120,5895632,7980152,5748871,9754064,11838032,5488559,...,n,y,n,n,n,n,n,n,n,1305
8,100008,2018-08-03 20:37:25,1744.8700,22402592,7066064,9151952,6920219,10998224,13083560,6732436,...,n,y,n,y,n,y,n,n,n,1937
9,100009,2018-08-03 20:37:44,597.4580,33691800,10024720,12111408,9856772,13874960,15961096,9591811,...,n,y,n,n,n,y,n,n,n,2344


In [14]:
# Juliana 
rawtuxdata.query("AIC7XXX_BUILD_FIRMWARE == 'y'").sample(n=10)['cid'].values
#len(rawtuxdata.query("AIC7XXX_BUILD_FIRMWARE == 'y'").index)

# Djamel 
rawtuxdata.query("AIC79XX_BUILD_FIRMWARE == 'y'").sample(n=10)['cid'].values
len(rawtuxdata.query("AIC79XX_BUILD_FIRMWARE == 'y'").index)

# Arnaud 
#rawtuxdata.query("WANXL_BUILD_FIRMWARE == 'y' & (AIC7XXX_BUILD_FIRMWARE != 'y' & AIC79XX_BUILD_FIRMWARE != 'y')").sample(n=10)['cid'].values
#rawtuxdata.query("WANXL_BUILD_FIRMWARE == 'y' & (AIC7XXX_BUILD_FIRMWARE == 'y' | AIC79XX_BUILD_FIRMWARE == 'y')")['cid'].values
# rawtuxdata.query("WANXL_BUILD_FIRMWARE == 'y' & AIC7XXX_BUILD_FIRMWARE == 'y'")['cid'].values

# rawtuxdata.query("WANXL_BUILD_FIRMWARE == 'y'")['cid'].values

# Mathieu 
# GENERIC_ALLOCATOR == 'y' + DRM_VBOXVIDEO
# interestingly: 'm' (module) does not lead to a bug! < 1.5
# ['vmlinux', 'cid']
# rawtuxdata.query("GENERIC_ALLOCATOR == 'n' & DRM_VBOXVIDEO == 'y' & AIC7XXX_BUILD_FIRMWARE == 'y'")['cid'].values
# rawtuxdata.query("GENERIC_ALLOCATOR == 'n' & DRM_VBOXVIDEO == 'y' & WANXL_BUILD_FIRMWARE == 'y'")['cid'].values
rawtuxdata.query("GENERIC_ALLOCATOR == 'n' & DRM_VBOXVIDEO == 'y' & WANXL_BUILD_FIRMWARE != 'y' & AIC7XXX_BUILD_FIRMWARE != 'y' & AIC79XX_BUILD_FIRMWARE != 'y'")['cid'].values

ValueError: a must be greater than 0

In [15]:
# masking effect
# Mathieu
## is there a masking effect between?
#rawtuxdata.query("GENERIC_ALLOCATOR == 'n' & DRM_VBOXVIDEO == 'y' & AIC79XX_BUILD_FIRMWARE == 'y'")
## is there a masking effect between?
# rawtuxdata.query("GENERIC_ALLOCATOR == 'n' & DRM_VBOXVIDEO == 'y' & AIC7XXX_BUILD_FIRMWARE == 'y'")
## is there a masking effect between?
# rawtuxdata.query("GENERIC_ALLOCATOR == 'n' & DRM_VBOXVIDEO == 'y' & WANXL_BUILD_FIRMWARE == 'y'")
# rawtuxdata.query("GENERIC_ALLOCATOR == 'n' & DRM_VBOXVIDEO == 'y' & WANXL_BUILD_FIRMWARE != 'y' & AIC7XXX_BUILD_FIRMWARE != 'y' & AIC79XX_BUILD_FIRMWARE != 'y'")

In [16]:
rawtuxdata.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26730 entries, 0 to 26743
Columns: 12799 entries, cid to nbyes
dtypes: float64(24), int64(241), object(12534)
memory usage: 20.6 GB


In [17]:
# https://github.com/TuxML/ProjetIrma/issues/137
# We should check whether in the database we have configurations in which we have options values:
# CONFIG_WANXL_BUILD_FIRMWARE=y and CONFIG_PREVENT_FIRMWARE_BUILD=y
# Hypothesis: The decision tree showed that when CONFIG_WANXL_BUILD_FIRMWARE is activated, 
# there are 335 failures out of 337... 
# As discussed, the "two" exceptional cases causing failures were quite intriguing... And maybe you get the explanation! Strangely, and according to the decision tree, the activation of option "CONFIG_VIDEO_RCAR_VIN" avoids compilation failures. I suspect that incidentally it is a wrong explanation of non-compilation failures... and that in fact it is due to the presence of CONFIG_PREVENT_FIRMWARE_BUILD. I will verify this hypothesis!

#wanxl_configurations = rawtuxdata.query("WANXL_BUILD_FIRMWARE == 'y' & (AIC7XXX_BUILD_FIRMWARE != 'y' & AIC79XX_BUILD_FIRMWARE != 'y')")
#wanxl_configurations2 = rawtuxdata.query("WANXL_BUILD_FIRMWARE == 'y'")

# wanxl_configurations.query("vmlinux != -1") # not necessary a failure? 
#wanxl_configurations2.query("vmlinux != -1")

#rawtuxdata.query("PREVENT_FIRMWARE_BUILD == 'y' & WANXL_BUILD_FIRMWARE == 'y'") #.query("vmlinux != -1")

rawtuxdata.query("WANXL_BUILD_FIRMWARE == 'y'") #  & vmlinux != -1")
#rawtuxdata.query("VIDEO_RCAR_VIN == 'm'")



,cid,date,time,vmlinux,GZIP-bzImage,GZIP-vmlinux,GZIP,BZIP2-bzImage,BZIP2-vmlinux,BZIP2,...,APDS9960,ARCH_SUPPORTS_INT128,SLABINFO,MICROCODE_AMD,ISDN_DRV_HISAX,CHARGER_BQ24190,SND_SOC_NAU8825,BH1750,NETWORK_FILESYSTEMS,nbyes
26718,126719,2018-11-16 16:31:36,850.091,-1,-1,-1,-1,-1,-1,-1,...,m,y,n,n,n,n,n,n,n,1529


In [18]:
### basic stats about options and unique values 
## could be extended/improved 

tri_state_values = ['y', 'n', 'm']

ftuniques = []
freq_ymn_features = []
non_tristate_options = []

for col in rawtuxdata:
    ft = rawtuxdata[col]    
    # eg always "y"
    if len(ft.unique()) == 1:
        ftuniques.append(col)
    # only tri-state values (y, n, m) (possible TODO: handle numerical/string options)    
    elif all(x in tri_state_values for x in ft.unique()):     #len(ft.unique()) == 3: 
        freq = ft.value_counts(normalize=True)
        freqy = 0
        freqn = 0
        freqm = 0
        if ('y' in freq.index):
            freqy = freq['y']
        if ('n' in freq.index):
            freqn = freq['n']
        if ('m' in freq.index):
            freqm = freq['m']
        freq_ymn_features.append((col, freqy, freqm, freqn))
    else:
        if not (col in size_methods): 
            non_tristate_options.append(col)
        

### TODO: we want to keep all quantitative values!
# non_tristate_options.remove('LZO') # ('vmlinux')

# we want to keep measurements (that are not tristate ;)) 
# non_tristate_options = list(set(non_tristate_options) - set(size_methods))

#### print options with unique values
# options with only one value eg always "y"
#i = 0
#for ft in ftuniques:
#    print(ft + " (" + str(i) + ")")
#    i = i + 1

print("Original size (#configs/#options) of the dataset " + str(rawtuxdata.shape))
print ("Number of options with only one value (eg always y): " + str(pd.DataFrame(ftuniques).shape))

# maybe we can drop options with only one unique value (no interest for machine learning)
# TODO: maybe we can rely on more traditional feature reduction techniques
# TODO: need to think about *when* to apply the removal 

#rawtuxdata.drop(columns=ftuniques,inplace=True) 

## non_tristate_options include basic stuff like date, time, cid but also string/numerical options
print ("Non tri-state value options (eg string or integer or hybrid values): " 
       + str(pd.DataFrame(non_tristate_options).shape) + " ") 
#      + str(pd.DataFrame(non_tristate_options)))


print ("Predictor variables: " + str(rawtuxdata.drop(columns=non_tristate_options).columns.size))
# frequency of y, m, and n values 
#plt.figure()
#pd.DataFrame(freq_ymn_features, columns=["feature", "freqy", "freqm", "freqn"]).plot(kind='hist', alpha=0.8) #plot()
#plt.show()


rawtuxdata.info(memory_usage='deep')

Original size (#configs/#options) of the dataset (26730, 12799)
Number of options with only one value (eg always y): (3415, 1)
Non tri-state value options (eg string or integer or hybrid values): (144, 1) 
Predictor variables: 12655
<class 'pandas.core.frame.DataFrame'>
Int64Index: 26730 entries, 0 to 26743
Columns: 12799 entries, cid to nbyes
dtypes: float64(24), int64(241), object(12534)
memory usage: 20.6 GB


In [22]:
from sklearn.preprocessing import *
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor, GradientBoostingClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LassoCV
from sklearn.pipeline import Pipeline
from sklearn import linear_model
from sklearn import svm
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, balanced_accuracy_score
from sklearn import tree
import graphviz 


LEARN_COMPILATION_SUCCESS = True # costly in time and space 
compilation_status_column_name = 'compile_success'

def encode_data_compilation(rawtuxdata):
    lae = LabelEncoder()
    # we save quantitative values we want (here vmlinux, TODO: generalize)
    # the key idea is that the labelling encoder should not be applied to this kind of values (only to predictor variables!)
    # vml = rawtuxdata['LZO'] # rawtuxdata['vmlinux'] 
    o_sizes = rawtuxdata[size_methods]

    # we may remove non tri state options, but TODO there are perhaps some interesting options (numerical or string) here
    #tuxdata = rawtuxdata.drop(columns=non_tristate_options).drop(columns=['vmlinux']).apply(le.fit_transform)
    tuxdata_for_compilation = rawtuxdata.drop(columns=non_tristate_options).drop(columns=size_methods).apply(lae.fit_transform)

    #tuxdata['vmlinux'] = vml 
    tuxdata_for_compilation[size_methods] = o_sizes
    # we can ue vmlinux since it has been restored thanks to previous line
    tuxdata_for_compilation[compilation_status_column_name] = tuxdata_for_compilation['vmlinux'] != -1
    return tuxdata_for_compilation

def learn_compilation_success(tuxdata_for_compilation):
    TESTING_SIZE=0.001 # 0.001 
    X_train, X_test, y_train, y_test = train_test_split(tuxdata_for_compilation.drop(columns=size_methods).drop(columns=compilation_status_column_name), tuxdata_for_compilation[compilation_status_column_name], test_size=TESTING_SIZE, random_state=0)  
    clf = tree.DecisionTreeClassifier() #GradientBoostingClassifier(n_estimators=100) #RandomForestRegressor(n_estimators=100) #   #GradientBoostingRegressor(n_estimators=100)  
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    importances = clf.feature_importances_
    indices = np.argsort(importances)[::-1]    

    TOP_FT_IMPORTANCE=20
    print("Feature ranking: " + "top (" + str(TOP_FT_IMPORTANCE) + ")")
    for f in range(TOP_FT_IMPORTANCE): # len(indices)
        print("%d. feature %s %d (%f)" % (f + 1, tuxdata_for_compilation.columns[indices[f]], indices[f], importances[indices[f]]))
   
    
    dot_data = tree.export_graphviz(clf, out_file=None, 
                         feature_names=tuxdata_for_compilation.drop(columns=size_methods).drop(columns=compilation_status_column_name).columns,  
                         filled=True, rounded=True,
                         special_characters=True)  
    graph = graphviz.Source(dot_data)  
    graph.render("TUXML_compilation_failures")
    
    acc = accuracy_score (y_test, y_pred)
    prec = precision_score (y_test, y_pred)
    reca = recall_score (y_test, y_pred)
    f1 = f1_score (y_test, y_pred)
    balance_acc = balanced_accuracy_score (y_test, y_pred)
    print("Accuracy score:", acc)
    print("Precision score:", prec)
    print("Recall score:", reca)
    print("F1 score:", f1)
    print("Balance accuracy score:", balance_acc)

if (LEARN_COMPILATION_SUCCESS):
    tuxdata_for_compilation = encode_data_compilation(rawtuxdata)
    # tuxdata_for_compilation [compilation_status_column_name].describe() # TODO?
    learn_compilation_success(tuxdata_for_compilation)
    del tuxdata_for_compilation

Feature ranking: top (20)
1. feature DRM_VBOXVIDEO 8025 (0.765760)
2. feature GENERIC_ALLOCATOR 11938 (0.047757)
3. feature REGMAP_I2C 2242 (0.041473)
4. feature VIDEO_MUX 12574 (0.016087)
5. feature PINCTRL_UNIPHIER_LD4 1690 (0.014812)
6. feature VIDEO_SOLO6X10 5738 (0.010996)
7. feature WANXL_BUILD_FIRMWARE 10536 (0.008873)
8. feature UEVENT_HELPER_PATH 1365 (0.008797)
9. feature MTD_ESB2ROM 6978 (0.008744)
10. feature COMEDI_DT282X 4391 (0.008545)
11. feature REGULATOR_MAX8907 2927 (0.008364)
12. feature BACKLIGHT_SAHARA 2158 (0.008295)
13. feature 60XX_WDT 2812 (0.008147)
14. feature XEN_FBDEV_FRONTEND 10710 (0.007618)
15. feature HX711 6211 (0.007406)
16. feature LEDS_LM355x 7834 (0.006665)
17. feature VIDEO_XILINX_VTC 1252 (0.004655)
18. feature GPIO_WATCHDOG 11681 (0.004444)
19. feature USB_GSPCA_OV534 8958 (0.004311)
20. feature VIDEO_EXYNOS4_FIMC_IS 460 (0.002948)
Accuracy score: 1.0
Precision score: 1.0
Recall score: 1.0
F1 score: 1.0
Balance accuracy score: 1.0


In [21]:
### CRC32 == m?
# VIDEO_OV5640 == 'y'? 
# rawtuxdata[['VIDEO_V4L2', 'MEDIA_TUNER']]
# & MEDIA_TUNER != 'n' 
# VIDEO_V4L2 != 'n' & 
# V4L2_FWNODE
rawtuxdata.query("VIDEO_MUX == 'y' & VIDEO_OV5640 == 'm' & CRC32 != 'n' & UBSAN_SANITIZE_ALL != 'y' & GENERIC_ALLOCATOR != 'n' & WANXL_BUILD_FIRMWARE != 'y' & AIC7XXX_BUILD_FIRMWARE != 'y' & AIC79XX_BUILD_FIRMWARE != 'y' & vmlinux == -1")[['VIDEO_V4L2', 'MEDIA_TUNER', 'V4L2_FWNODE', 'vmlinux']] # ['cid'].values
#rawtuxdata.query("VIDEO_V4L2 == 'm'")[['VIDEO_V4L2', 'MEDIA_TUNER', 'vmlinux']]

,VIDEO_V4L2,MEDIA_TUNER,V4L2_FWNODE,vmlinux
390,m,m,m,-1
1698,m,m,m,-1
8827,m,n,m,-1
19021,m,m,m,-1
19060,m,m,m,-1
22676,m,m,m,-1


In [41]:
print("specific analysis for dataset 100K-126K")
# [['VIDEO_V4L2', 'MEDIA_TUNER', 'V4L2_FWNODE', 'vmlinux']] # 
rawtuxdata.query("REGMAP_I2C == 'm' & VIDEO_MUX == 'y' & VIDEO_XILINX_VTC == 'm' & GENERIC_ALLOCATOR == 'y' & WANXL_BUILD_FIRMWARE == 'n'")[['cid', 'VIDEO_V4L2', 'MEDIA_TUNER', 'V4L2_FWNODE', 'vmlinux']] # ['cid'].values
#

specific analysis for dataset 100K-126K


,cid,VIDEO_V4L2,MEDIA_TUNER,V4L2_FWNODE,vmlinux
390,100390,m,m,m,-1
1698,101698,m,m,m,-1
6362,106362,m,m,m,-1
8827,108827,m,n,m,-1
19021,119021,m,m,m,-1
19060,119060,m,m,m,-1
22676,122676,m,m,m,-1


In [58]:
print("specific analysis for dataset 100K-126K")
# [['VIDEO_V4L2', 'MEDIA_TUNER', 'V4L2_FWNODE', 'vmlinux']] # 
rawtuxdata.query("VIDEO_MUX != 'y' & VIDEO_XILINX_VTC == 'm' & GENERIC_ALLOCATOR == 'y' & WANXL_BUILD_FIRMWARE == 'n' & vmlinux == -1")[['cid', 'vmlinux', 'SND_ATMEL_SOC_CLASSD', 'SND_ATMEL_SOC_DMA', 'SND_ATMEL_SOC_SSC_DMA', 'SND_SOC_GENERIC_DMAENGINE_PCM']]
#['cid'].values
# [['cid', 'VIDEO_V4L2', 'MEDIA_TUNER', 'V4L2_FWNODE', 'vmlinux', 'UEVENT_HELPER_PATH']] # ['cid'].values
#

specific analysis for dataset 100K-126K


,cid,vmlinux,SND_ATMEL_SOC_CLASSD,SND_ATMEL_SOC_DMA,SND_ATMEL_SOC_SSC_DMA,SND_SOC_GENERIC_DMAENGINE_PCM
8420,108420,-1,m,m,n,y


In [63]:
print("specific analysis for dataset 100K-126K")
rawtuxdata.query("GENERIC_ALLOCATOR == 'n' & DRM_VBOXVIDEO == 'y'")[['cid', 'vmlinux']]['cid'].values
#

specific analysis for dataset 100K-126K


array([100367, 100410, 100459, 100650, 101162, 101542, 101662, 101815,
       102331, 102572, 102861, 103129, 103843, 104114, 104747, 105025,
       105093, 105755, 106373, 106881, 106925, 106993, 107103, 107209,
       107483, 107653, 107821, 107901, 107950, 108175, 108652, 109208,
       109285, 109460, 109782, 109815, 109827, 109997, 110204, 110363,
       110608, 110755, 111024, 111207, 111279, 111795, 112063, 112172,
       113184, 113473, 113695, 113825, 113872, 113928, 114243, 114403,
       115503, 115574, 115731, 115800, 116165, 116550, 116896, 117277,
       117360, 117442, 117780, 118096, 118563, 118808, 118947, 119109,
       119590, 119806, 119828, 120205, 120411, 120603, 121270, 123103,
       123346, 123348, 123387, 123536, 124344, 124690, 125121, 125230,
       125374, 125567, 125969, 126432])

In [71]:
print("specific analysis for dataset 100K-126K")
rawtuxdata.query("BACKLIGHT_SAHARA == 'm' & RT_MUTEXES == 'n' & GENERIC_ALLOCATOR == 'n' & DRM_VBOXVIDEO != 'y'")[['cid', 'vmlinux', 'USB_COMMON', 'USB_GADGET', 'USB', 'USB_ULPI_BUS']]

specific analysis for dataset 100K-126K


,cid,vmlinux,USB_COMMON,USB_GADGET,USB,USB_ULPI_BUS
14662,114662,-1,y,m,m,y


In [86]:
print("specific analysis for dataset 100K-126K")
# PINCTRL_UNIPHIER_LD4 == 'y' & 
rawtuxdata.query("VIDEO_EXYNOS4_FIMC_IS == 'm' & VIDEO_XILINX_VTC == 'n' & WANXL_BUILD_FIRMWARE != 'y' & GENERIC_ALLOCATOR == 'y'")[['cid', 'vmlinux', 'VIDEO_MUX', 'VIDEO_V4L2', 'PINCTRL_UNIPHIER_LD4']]
# ['cid'].values
#[['cid', 'vmlinux']]

specific analysis for dataset 100K-126K


,cid,vmlinux,VIDEO_MUX,VIDEO_V4L2,PINCTRL_UNIPHIER_LD4
3765,103765,24156792,m,m,n
5958,105958,20028976,m,m,n
6470,106470,25364608,n,y,n
6812,106812,31756560,m,m,n
11538,111538,37598568,m,y,n
13731,113731,-1,y,m,y
14071,114071,19240848,n,m,n
14477,114477,26603568,m,m,n
15406,115406,22837656,m,y,n
24183,124183,20201888,m,y,n


In [97]:
print("specific analysis for dataset 100K-126K")
# PINCTRL_UNIPHIER_LD4 == 'y' & 
rawtuxdata.query("VIDEO_EXYNOS4_FIMC_IS != 'y' & VIDEO_XILINX_VTC != 'y' & WANXL_BUILD_FIRMWARE != 'y' & GENERIC_ALLOCATOR != 'n' & vmlinux == -1")['cid'].values
# [['cid', 'vmlinux', 'VIDEO_MUX', 'VIDEO_V4L2']]
#

specific analysis for dataset 100K-126K


array([100390, 100734, 101561, 101698, 106362, 106779, 108420, 108827,
       109530, 112930, 113286, 113731, 115367, 119021, 119060, 121405,
       122676, 124965, 126652])

In [98]:
rawtuxdata.query("cid == 109530")[['ACPI', 'BACKLIGHT_CLASS_DEVICE']] 

,ACPI,BACKLIGHT_CLASS_DEVICE
9530,n,n


In [107]:
rawtuxdata.query("cid == 115367")[['cid', 'vmlinux', 'DRM_VBOXVIDEO', 'DRM', 'PCI', 'DRM_KMS_HELPER']]


,cid,vmlinux,DRM_VBOXVIDEO,DRM,PCI,DRM_KMS_HELPER
15367,115367,-1,y,y,y,y


In [101]:
rawtuxdata.query("cid == 100734 | cid == 101561 | cid == 126652 | cid == 113286")[['cid', 'vmlinux', 'ACPI', 'BACKLIGHT_CLASS_DEVICE']]

,cid,vmlinux,ACPI,BACKLIGHT_CLASS_DEVICE
734,100734,-1,n,m
1561,101561,-1,n,m
13286,113286,-1,n,m
26651,126652,-1,n,m


In [109]:
rawtuxdata.query(" | ".join(["cid == " + str(e) for e in [100390, 101698, 106362, 106779, 108827, 113731, 119021, 119060, 122676, 124965]]))[['cid', 'vmlinux', 'MEDIA_TUNER', 'VIDEO_MUX', 'VIDEO_V4L2']]

,cid,vmlinux,MEDIA_TUNER,VIDEO_MUX,VIDEO_V4L2
390,100390,-1,m,y,m
1698,101698,-1,m,y,m
6362,106362,-1,m,y,m
6779,106779,-1,n,y,m
8827,108827,-1,n,y,m
13731,113731,-1,m,y,m
19021,119021,-1,m,y,m
19060,119060,-1,m,y,m
22676,122676,-1,m,y,m
24965,124965,-1,m,y,m


In [105]:
rawtuxdata.query("cid == 121405")[['JOLIET', 'NLS_DEFAULT', 'NLS']]

,JOLIET,NLS_DEFAULT,NLS
21405,n,NaN,n


In [20]:
# 
# 
# IPV6 == 'n'
# rawtuxdata.query("(UBSAN_ALIGNMENT != 'n' | UBSAN_NULL != 'n') & IPV6 != 'm' & FORTIFY_SOURCE == 'y' & IPV6 != 'y' & INFINIBAND_ADDR_TRANS == 'y' & UBSAN_SANITIZE_ALL == 'y' & GENERIC_ALLOCATOR != 'n' & WANXL_BUILD_FIRMWARE != 'y' & AIC7XXX_BUILD_FIRMWARE != 'y' & AIC79XX_BUILD_FIRMWARE != 'y'")[['IPV6', 'UBSAN_ALIGNMENT', 'UBSAN_NULL', 'vmlinux']]  # ['cid'].values 
rawtuxdata.query("IPV6 == 'n' & (UBSAN_ALIGNMENT != 'n' | UBSAN_NULL != 'n') & FORTIFY_SOURCE == 'y' & INFINIBAND_ADDR_TRANS == 'y' & UBSAN_SANITIZE_ALL == 'y' & GENERIC_ALLOCATOR != 'n' & WANXL_BUILD_FIRMWARE != 'y' & AIC7XXX_BUILD_FIRMWARE != 'y' & AIC79XX_BUILD_FIRMWARE != 'y'")[['IPV6', 'UBSAN_ALIGNMENT', 'UBSAN_NULL', 'vmlinux']]  # ['cid'].values 


,IPV6,UBSAN_ALIGNMENT,UBSAN_NULL,vmlinux


In [ ]:
errors = pd.read_csv('errors.csv')
clusters = []

#rawtuxdata.query("vmlinux == -1", inplace=True) 
#print (len(rawtuxdata))
#rawtuxdata['cluster'] = 'null'
#num = len(rawtuxdata.columns) - 1 
# query("vmlinux == -1").
i = 0
for row in rawtuxdata.iterrows():
    cid = row[1]['cid']
    if (row[1]['vmlinux'] != -1):
        log = 'NO_FAILURE'
    else:
        cid_error = errors.query("cid == " + str(cid))
        if (len(cid_error) > 0):
            log = cid_error['error'].iloc[0]
        else:
            print("strange case (no cid?)")
    clusters.append(log)
    #rawtuxdata.iloc[i, num] = log
#    i = i + 1
#rawtuxdata[:10]
#pd.merge(rawtuxdata, errors, on='cid', how='outer')

In [ ]:
rawtuxdata['cluster_error'] = clusters

In [ ]:
rawtuxdata.query("cluster_error != 'NO_FAILURE'")[['cid', 'cluster_error', 'vmlinux']][:10]

In [ ]:
rawtuxdata.query("vmlinux == -1").shape, rawtuxdata.query("cluster_error != 'NO_FAILURE'").shape

In [ ]:
rawtuxdata.query("AIC7XXX_BUILD_FIRMWARE == 'y'")['cluster_error'].unique()

In [ ]:
rawtuxdata.query("IPV6 == 'n' & (UBSAN_ALIGNMENT != 'n' | UBSAN_NULL != 'n') & FORTIFY_SOURCE == 'y' & INFINIBAND_ADDR_TRANS == 'y' & UBSAN_SANITIZE_ALL == 'y' & GENERIC_ALLOCATOR != 'n' & WANXL_BUILD_FIRMWARE != 'y' & AIC7XXX_BUILD_FIRMWARE != 'y' & AIC79XX_BUILD_FIRMWARE != 'y'")['cluster_error'].unique()
#[['IPV6', 'UBSAN_ALIGNMENT', 'UBSAN_NULL', 'vmlinux']]  # ['cid'].values 


In [ ]:
#clusters = pd.read_csv('clusters.csv')
#clusters

In [ ]:
rawtuxdata.query("MCP4725 == 'y' & CRC32 != 'y' & UBSAN_SANITIZE_ALL == 'n' & GENERIC_ALLOCATOR != 'n' & WANXL_BUILD_FIRMWARE != 'y' & AIC7XXX_BUILD_FIRMWARE != 'y' & AIC79XX_BUILD_FIRMWARE != 'y'")[['cid', 'vmlinux', 'CRC32', 'MCP4725', 'DRM_I915_GVT', 'cluster_error']]

In [ ]:
rawtuxdata.query("cluster_error == 'L6'") 

In [ ]:
# ['cid'].values # 
rawtuxdata.query("SCSI_SRP_ATTRS == 'n' & INFINIBAND_SRPT == 'y' & ISDN_DIVAS_USERIDI == 'n' & MLX5_INFINIBAND != 'y' & GENERIC_ALLOCATOR == 'n' & DRM_VBOXVIDEO != 'y' & WANXL_BUILD_FIRMWARE != 'y' & AIC7XXX_BUILD_FIRMWARE != 'y' & AIC79XX_BUILD_FIRMWARE != 'y'")[['cid', 'vmlinux', 'IPV6', 'UBSAN_SANITIZE_ALL', 'INFINIBAND_ADDR_TRANS', 'FORTIFY_SOURCE', 'UBSAN_ALIGNMENT', 'UBSAN_NULL']]

In [ ]:
rawtuxdata.query("MLX5_INFINIBAND == 'm' & GENERIC_ALLOCATOR == 'n' & DRM_VBOXVIDEO != 'y' & WANXL_BUILD_FIRMWARE != 'y' & AIC7XXX_BUILD_FIRMWARE != 'y' & AIC79XX_BUILD_FIRMWARE != 'y'")[['cid', 'vmlinux', 'IPV6', 'UBSAN_SANITIZE_ALL', 'INFINIBAND_ADDR_TRANS', 'FORTIFY_SOURCE', 'UBSAN_ALIGNMENT', 'UBSAN_NULL', 'NET_FOU_IP_TUNNELS']]

In [ ]:
rawtuxdata.query("GPIO_GRGPIO == 'm' & ISDN_DIVAS_USERIDI == 'm' & MLX5_INFINIBAND == 'n' & GENERIC_ALLOCATOR == 'n' & DRM_VBOXVIDEO != 'y' & WANXL_BUILD_FIRMWARE != 'y' & AIC7XXX_BUILD_FIRMWARE != 'y' & AIC79XX_BUILD_FIRMWARE != 'y'")[['cid', 'vmlinux', 'IPV6', 'UBSAN_SANITIZE_ALL', 'INFINIBAND_ADDR_TRANS', 'FORTIFY_SOURCE', 'UBSAN_ALIGNMENT', 'UBSAN_NULL', 'NET_FOU_IP_TUNNELS']]

In [ ]:
# ['cid'].values # 
rawtuxdata.query("GENERIC_ALLOCATOR == 'n' & DRM_VBOXVIDEO != 'y' & WANXL_BUILD_FIRMWARE != 'y' & AIC7XXX_BUILD_FIRMWARE != 'y' & AIC79XX_BUILD_FIRMWARE != 'y' & vmlinux == -1")[['cid', 'vmlinux', 'IPV6', 'UBSAN_SANITIZE_ALL', 'INFINIBAND_ADDR_TRANS', 'FORTIFY_SOURCE', 'UBSAN_ALIGNMENT', 'UBSAN_NULL', 'NET_FOU_IP_TUNNELS', 'MLX5_INFINIBAND']]

In [ ]:
rawtuxdata.query("MLX5_INFINIBAND == 'y' & GENERIC_ALLOCATOR == 'n' & DRM_VBOXVIDEO != 'y' & WANXL_BUILD_FIRMWARE != 'y' & AIC7XXX_BUILD_FIRMWARE != 'y' & AIC79XX_BUILD_FIRMWARE != 'y'")[['cid', 'vmlinux', 'IPV6', 'UBSAN_SANITIZE_ALL', 'INFINIBAND_ADDR_TRANS', 'FORTIFY_SOURCE', 'UBSAN_ALIGNMENT', 'UBSAN_NULL', 'NET_FOU_IP_TUNNELS', 'MLX5_INFINIBAND']]


In [ ]:
rawtuxdata.query("cid == 65670")[['ACPI', 'BACKLIGHT_CLASS_DEVICE', 'DRM_PANEL_SAMSUNG_S6E3HA2', 'DRM_I915']]

In [ ]:
#rawtuxdata.query("vmlinux == -1 & XEN_PCIDEV_BACKEND == 'n' & WIL6210_TRACING == 'n' & INFINIBAND_SRPT == 'n' & ISDN_DIVAS_USERIDI == 'n' & MLX5_INFINIBAND != 'y' & GENERIC_ALLOCATOR == 'n' & DRM_VBOXVIDEO != 'y' & WANXL_BUILD_FIRMWARE != 'y' & AIC7XXX_BUILD_FIRMWARE != 'y' & AIC79XX_BUILD_FIRMWARE != 'y' ")[['cid', 'vmlinux', 'IPV6', 'UBSAN_SANITIZE_ALL', 'INFINIBAND_ADDR_TRANS', 'FORTIFY_SOURCE', 'UBSAN_ALIGNMENT', 'UBSAN_NULL', 'NET_FOU_IP_TUNNELS', 'MLX5_INFINIBAND', 'SCSI_SRP_ATTRS', 'RDS']]